In [36]:
import sys
sys.path.insert(0, '../')
import os
import pygrib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from joblib import load
from GRIBFunctions import GRIBProcessor
from GRIBFunctions import GRIBPlotter
import xarray as xr


class WeatherPredictor:
    def __init__(self, model_file):
        self.model = load(model_file)
        self.prediction = None
        self.lats = None
        self.lons = None

        
    def extract_data_and_coords(self, filename):
        with pygrib.open(filename) as grbs:
            grb = grbs.select()[0]
            data = grb.values
            self.lats, self.lons = grb.latlons()
            return data.flatten()

        
    def make_prediction(self, grib_file):
        X_new = self.extract_data_and_coords(grib_file)
        X_new = X_new.reshape(-1, 1)
        self.prediction = self.model.predict(X_new).reshape(361, 720)
        

    def output_csv(self):
        meteo_arr = xr.DataArray(self.prediction, dims=['latitude', 'longitude'])
        csv_file = GRIBProcessor.csv_conversion(meteo_arr)
        return csv_file
    
    
    def output_plot(self):
        xarray_data = xr.DataArray(self.prediction, dims=['latitude', 'longitude'])
        plot_file = GRIBPlotter.plot_grib(xarray_data)
        return plot_file        
    


In [37]:
predictor = WeatherPredictor('regr_model.joblib')
predictor.make_prediction('../model-testing/new-twelve')
csv_file = predictor.output_csv()
plot_file = predictor.output_plot()


ValueError: cannot rename 'gh' because it is not a variable or dimension in this dataset